최종 GUI

In [7]:
import tkinter as tk
from tkinter import ttk
import mysql.connector
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from tqdm import tqdm 
import re
import mysql.connector
from selenium.webdriver.common.keys import Keys
import requests
import pandas as pd
from PIL import Image, ImageTk
from io import BytesIO
from threading import Thread
import tkinter.messagebox as messagebox

def category(option_cat):
    
    type = option_cat

    if type == '스킨/토너':
        types = 'skin'

    elif type == '세럼/앰플':
        types = 'essence'
    
    elif type == '로션':
        types = 'lotion'

    elif type == '크림':
        types = 'cream'
    
    else:
        types = 'error'
    
    try:
        data = pd.read_csv("/home/addinedu/dev_ws/data/df/new_{}.csv".format(types))
    except:
        print('데이터프레임 못갖고옴')
    
    return data


def show_image(image):
    # 이미지를 Tkinter PhotoImage 객체로 변환
    photo = ImageTk.PhotoImage(image)
        
    # 이미지 레이블 업데이트
    image_label.config(image=photo)
    image_label.image = photo 

def on_key(event):
    if event.char == 'c':
        root.quit()
        root.destroy()

def op(options):
    if options[0] == '지성':
        f_col = '지성'
        f_con_value = 20
    elif options[0] == '건성':
        f_col = '건성'
        f_con_value = 40
    elif options[0] == '복합성':
        f_col = '복합성'
        f_con_value = 50
    elif options[0] == '전체':
        f_col = '지성'
        f_con_value = 0


    if options[2] == '보습':
        s_col = '보습'
        s_con_value = 65
    elif options[2] == '진정':
        s_col = '진정'
        s_con_value = 70
    elif options[2] == '주름/미백':
        s_col = '주름/미백'
        s_con_value = 40
    elif options[2] == '전체':
        s_col = '보습'
        s_con_value = 0


    if options[3] == '자극없는':
        th_col = '자극x'
        th_con_value = 75
    elif options[3] == '자극있는':
        th_col = '자극O'
        th_con_value = 10
    elif options[3] == '보통':
        th_col = '보통'
        th_con_value = 30
    elif options[3] == '전체':
        th_col = '보통'
        th_con_value = 0   

    #v1 <= 가격 <= v2
    if options[4] == '~15000':
        cal = ''
        f_col = '세일가'
        f_con_value1 = 0
        f_con_value2 = 15000
    elif options[4] == '15000~25000':
        cal = '<='
        f_col = '세일가'
        f_con_value1 = 15000
        f_con_value2 = 25000
    elif options[4] == '25000~35000':
        cal = '<='
        f_col = '세일가'
        f_con_value1 = 25000
        f_con_value2 = 35000
    elif options[4] == '35000~':
        cal = '<='
        f_col = '세일가'
        f_con_value1 = 35000
        f_con_value2 = 1000000
    elif options[4] == '전체':
        f_col = '세일가'
        cal = '<='
        f_con_value = 0
        f_con_value2 = 1000000 

    option_cat = options[5]
    data = category(option_cat)
    result_series = data.loc[(data[f_col] >= f_con_value) & (data[s_col] >= s_con_value) & (data[th_col] >= th_con_value) & (data[f_col].between(f_con_value, f_con_value2)) & (data['평점 갯수'] >= 1000), '제품명']

    print(result_series)
    return result_series

def img_scrape(result):
    url = "https://www.oliveyoung.co.kr/store/main/main.do?oy=0"
    
    chrome_options = Options()
    chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome(service=Service("/home/addinedu/dev_ws/driver/chromedriver-linux64/chromedriver"), options=chrome_options)

    driver. get(url)
    search1 = driver.find_element(By.XPATH, '//*[@id="w_search_box"]/div[1]/label')
    search1.click()
    time.sleep(1)
    search = driver.find_element(By.CSS_SELECTOR, '#query')
    # search.clear()
    search.send_keys(result)
    search.send_keys(Keys.ENTER)


    req = driver.page_source
    soup = BeautifulSoup(req,'html.parser') 

    # 이미지 태그를 찾음
    img = soup.find(id = 'w_cate_prd_list')
    img1 = img.find('img')['src']
    response = requests.get(img1)

    if response.status_code == 200:
        image_data = response.content
        image = Image.open(BytesIO(image_data))
        image.thumbnail((400, 400))   
        print(result)
    else:
        print("이미지를 가져오는데 실패했습니다.")

    return image

def scrape_and_display_image(options):

    result_series = op(options)
    # result_series = result_series.str.replace(r'\[[^\]]*\]|\([^)]*\)', '', regex=True)
    # result_series = result_series.str.replace(r'\[[^\]]*\]', '', regex=True)
    print('result_series:',result_series)

    
    result = result_series.iloc[0] 
    print('result:',result)

    image = img_scrape(result)
    
    # 이미지를 표시할 라벨 생성
    image_label = tk.Label(root)
    image_label.grid(row=5, column=0, columnspan=3, padx=4, pady=5)
    
    # 이미지를 Tkinter PhotoImage 객체로 변환
    photo = ImageTk.PhotoImage(image)
    
    # 라벨에 이미지 표시
    image_label.config(image=photo)
    image_label.image = photo    
    # lab.config(text=str(options))
    
    info_text = tk.Text(root, height=20, width=40)
    info_text.grid(row=5, column=3, columnspan=3, padx=4, pady=1)  # 텍스트 상자를 아래쪽에 배치
    
    info_text2 = tk.Text(root, height=1, width=40)
    info_text2.grid(row=4, column=2, columnspan=3, padx=4, pady=20)  # 텍스트 상자를 아래쪽에 배치
    
    # 이미지에 대한 정보를 텍스트 상자에 추가
    
    data = category(options[5]) 

    info = data[data['제품명'] == result]
    gradients = info['성분'].iloc[0]
    price = info['세일가'].iloc[0]

    # info = 
    info_text.insert(tk.END, f" 가격: {price} \n \n 성분: {gradients}")
    info_text2.insert(tk.END, f"제품명: {result}")

    # 프로그래스 바 멈춤
    progress_bar.stop()


options = []

t = None

def click():
    global options

    progress_bar.start(10)
    
    text = combobox1.get()
    text2 = combobox2.get()
    text3 = combobox3.get()
    text4 = combobox4.get()
    text5 = combobox5.get()
    text6 = combobox6.get()

    options1.append(text)
    options2.append(text2)
    options3.append(text3)
    options4.append(text4)
    options5.append(text5)
    options6.append(text6)

    options.append(options1[0])
    options.append(options2[0])
    options.append(options3[0])
    options.append(options4[0])
    options.append(options5[0])
    options.append(options6[0])

    print(options)

    t = Thread(target=scrape_and_display_image, args=(options,))
    t.start()
    



# def reset_and_restart():
#     # 새로운 스레드에서 reset_gui 함수 호출
#     t = Thread(target=reset_gui)
#     t.start()

# def reset_gui():
#     # 콤보박스 초기화
#     combobox1.current(0)
#     combobox2.current(0)
#     combobox3.current(0)
#     combobox4.current(0)
#     combobox5.current(0)
#     combobox6.current(0)

#     # 이미지 라벨 초기화
#     image_label.config(image=None)

#     # 스레드에게 위젯 제거 요청
#     root.after(0, remove_widgets)

# def remove_widgets():
#     # 텍스트 상자 제거
#     info_text.grid_forget()
#     info_text2.grid_forget()

#     # 이미지 상자 제거
#     image_label.grid_forget()





options1 = []
options2 = []
options3 = []
options4 = []
options5 = []
options6 = []

# Tkinter 창 생성
root = tk.Tk()
root.title("스킨케어 추천")
root.geometry("800x600")


# 설명 레이블 생성
label1 = tk.Label(root, text="피부 타입(선택)")
label1.grid(row=0, column=1, padx=5, pady=5)

label2 = tk.Label(root, text="피부 상태(선택)")
label2.grid(row=0, column=2, padx=5, pady=5)

label3 = tk.Label(root, text="효과(선택)")
label3.grid(row=0, column=3, padx=5, pady=5)

label4 = tk.Label(root, text="민감도(선택)")
label4.grid(row=0, column=4, padx=5, pady=5)

label5 = tk.Label(root, text="가격(선택)")
label5.grid(row=0, column=5, padx=5, pady=5)

label6 = tk.Label(root, text="카테고리(필수*)")
label6.grid(row=0, column=0, padx=5, pady=5)


# Combobox 생성
values1 = ["전체","지성", "건성", "복합성"]
values2 = ["전체","트러블", "홍조", "모공"]
values3 = ["전체","보습", "진정", "주름/미백"]
values4 = ["전체","자극없는", "보통", "자극있는"]
values5 = ["전체","~15000", "15000~25000", "25000~35000", "35000~"]
values6 = ['전체','스킨/토너','세럼/앰플', '로션', '크림']

combobox1 = ttk.Combobox(root, values=values1, width=8)  
combobox1.grid(row=1, column=1, padx=8, pady=5)
combobox1.current(0)

combobox2 = ttk.Combobox(root, values=values2, width=8)  
combobox2.grid(row=1, column=2, padx=8, pady=5)
combobox2.current(0)

combobox3 = ttk.Combobox(root, values=values3, width=8)  
combobox3.grid(row=1, column=3, padx=8, pady=5)
combobox3.current(0)

combobox4 = ttk.Combobox(root, values=values4, width=8)  
combobox4.grid(row=1, column=4, padx=8, pady=5)
combobox4.current(0)

combobox5 = ttk.Combobox(root, values=values5, width=12)  
combobox5.grid(row=1, column=5, padx=8, pady=5)
combobox5.current(0)

combobox6 = ttk.Combobox(root, values=values6, width=12)  
combobox6.grid(row=1, column=0, padx=8, pady=5)
combobox6.current(0)

progress_bar = ttk.Progressbar(root, orient="horizontal", length=200, mode="indeterminate")
progress_bar.grid(row=2, column=0, columnspan=6, padx=8, pady=5)

btn = tk.Button(root, text="조회", command=click)
btn.grid(row=1, column=6, columnspan=4, padx=8, pady=5)

# reset_btn = tk.Button(root, text="초기화", command=reset_and_restart)
# reset_btn.grid(row=2, column=6, columnspan=4, padx=8, pady=5)

# 선택된 옵션을 표시할 레이블 생성
lab = tk.Label(root, text="")
lab.grid(row=2, column=0, columnspan=4)


# 'c' 키 입력 시 종료하는 이벤트 바인딩
root.bind("<Key>", on_key)

# Tkinter 창 실행
root.mainloop()

['지성', '트러블', '보습', '자극없는', '25000~35000', '스킨/토너']
0                        바이오더마 하이드라비오 토너 500ml
8                        라네즈 크림스킨 170ml 리필 기획 
16                  에스네이처 아쿠아 오아시스 토너 300ml 기획
22                          라네즈 크림스킨 170ml 기획 
146     더랩바이블랑두 저분자 히알루론산 딥 토너 200ml + 50ml 기획
171                코스알엑스 풀핏 프로폴리스 시너지 토너 280ml
172                       이즈앤트리 히아루론산 토너 200ml
187                           하다라보 고쿠쥰 로션170ml
198                      이즈앤트리 히아루론산 토너 더블 기획 
214                      바이오더마 하이드라비오 토너 250ml
233                          유리아쥬 제모스 토너 500ml
246                       토니모리 원더 세라마이드 모찌 토너 
Name: 제품명, dtype: object
result_series: 0                        바이오더마 하이드라비오 토너 500ml
8                        라네즈 크림스킨 170ml 리필 기획 
16                  에스네이처 아쿠아 오아시스 토너 300ml 기획
22                          라네즈 크림스킨 170ml 기획 
146     더랩바이블랑두 저분자 히알루론산 딥 토너 200ml + 50ml 기획
171                코스알엑스 풀핏 프로폴리스 시너지 토너 280ml
172                       이즈앤트리 히아루론산 토너 200ml
187            

In [ ]:
# types = 'essence'
data = pd.read_csv("/home/addinedu/dev_ws/data/df/new_skin.csv")
result ='바이오더마 하이드라비오 토너 500ml'
product_row = data[data['제품명'] == result]
product_row['성분'].iloc[0]

이미지 크기조절

In [ ]:
# from PIL import Image

# def resize_image(input_image_path, output_image_path, new_size):
#     # 이미지 열기
#     image = Image.open(input_image_path)
#     # 크기 조절
#     resized_image = image.resize(new_size)
#     # 크기 조절된 이미지 저장
#     resized_image.save(output_image_path)

# # 입력 이미지 경로와 출력 이미지 경로 정의
# input_image_path = "/home/addinedu/Pictures/Screenshots/unnamed.png"
# output_image_path = "/home/addinedu/Pictures/Screenshots/unnamed2.png"

# # 새로운 이미지 크기 정의 (너비, 높이)
# new_size = (600, 600)

# # 이미지 크기 조절
# resize_image(input_image_path, output_image_path, new_size)